In [1]:
import pandas as pd
import numpy as np
from osgeo import gdal
from gdalconst import *
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_rows', 20000)
pd.set_option('display.max_columns', 20000)
pd.set_option('display.max_colwidth', -1)
import warnings
warnings.filterwarnings("ignore")

In /home/estefania/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/estefania/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/estefania/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In /home/estefania/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/estefania/.local/lib/python3.6/site-packages/matplo

In [3]:
#directoriosancor="/home/manuela/sancor-2.0/" #MANU
directoriosancor="/home/estefania/Desktop/sancor-2.0/"   #EDU

directoriopuntos=directoriosancor+"1CSV/Agua/"
directoriocsvs=directoriosancor+"1CSV/"

In [4]:
# VERA JUAN PABLO (correr solamente 1 de las 3 celdas)
df_puntos_agua = pd.read_csv(directoriopuntos+"AguaVera21022019_trans.csv")
directoriolectura=directoriocsvs+"Vera_csvs/"
prefijolectura="Vera_"
rangodefilas=range(54)          #estas son las filas cuyos csv queremos crear

totaldefilas=54        #cantidaddefilas de df_puntos_agua
longitoo=[]
latitoo=[]  
for r in range(totaldefilas):
    longitoo.append(df_puntos_agua.Long_trans[r])
    latitoo.append(df_puntos_agua.Lat_trans[r])

In [4]:
# NICOLÁS AGUA NO PERMANENTE
df_puntos_agua = pd.read_csv(directoriopuntos+"aguanopermanente_filtrada.csv")
directoriolectura=directoriocsvs+"nico_agua_no_permanente_csvs/"
prefijolectura="aguanoperm_"

rangodefilas=range(29)          #estas son las filas cuyos csv queremos crear

totaldefilas=29
longitoo=[]
latitoo=[]
for r in range(totaldefilas):
    longitoo.append(float(df_puntos_agua.Longitud[r].replace(",",".")))
    latitoo.append(float(df_puntos_agua.Latitud[r].replace(",",".")))

In [ ]:
# DIEGO (correr solamente 1 de las 3 celdas)
df_puntos_agua = pd.read_csv(directoriopuntos+"agua_2016_diego_100.csv")
directoriolectura=directoriocsvs+"diego_csvs/"
prefijolectura="diego_"
rangodefilas=range(100)          #estas son las filas cuyos csv queremos crear

totaldefilas=100        #cantidaddefilas de df_puntos_agua
longitoo=[]
latitoo=[]
for r in range(totaldefilas):
    longitoo.append(float(df_puntos_agua.Long_trans[r].replace(",",".")))
    latitoo.append(float(df_puntos_agua.Lat_trans[r].replace(",",".")))

In [5]:
listadefechas = []
anios=range(2010,2020)
fechasporanio=range(1,365,8)
for yy in anios:
    for d in fechasporanio:
        listadefechas.append(str(yy)+"%03d"%d)

In [11]:
contador_mndwi=[]
for i in range(len(listadefechas)):
    contador_mndwi.append([])
    for j in rangodefilas:
        contador_mndwi[i].append(0)

In [12]:
contador_ndwi=[]
for i in range(len(listadefechas)):
    contador_ndwi.append([])
    for j in rangodefilas:
        contador_ndwi[i].append(0)

In [8]:
tol = 0.25

In [13]:
for e in rangodefilas:
    long=longitoo[e]
    lat=latitoo[e]
    for k in range(1):
        datos=pd.read_csv(directoriolectura+prefijolectura+ str(e).zfill(3)+"_mndwi_"+str(2*k+1)+"x"+str(2*k+1) +".csv")
        for i in range(1,len(datos.promedio) - 1):
            if abs(datos.promedio.iloc[i] - datos.promedio.iloc[i-1]) > tol and abs(datos.promedio.iloc[i] - datos.promedio.iloc[i+1]) > tol and (datos.promedio.iloc[i] - datos.promedio.iloc[i-1])*(datos.promedio.iloc[i] - datos.promedio.iloc[i+1]) > 0:
                #datos.promedio.iloc[i] = (datos.promedio.iloc[i-1] + datos.promedio.iloc[i+1])/2
                contador_mndwi[i][e] += 1     
                
        datos=pd.read_csv(directoriolectura+prefijolectura+ str(e).zfill(3)+"_ndwi_"+str(2*k+1)+"x"+str(2*k+1) +".csv")
        for i in range(1,len(datos.promedio) - 1):
            if abs(datos.promedio.iloc[i] - datos.promedio.iloc[i-1]) > tol and abs(datos.promedio.iloc[i] - datos.promedio.iloc[i+1]) > tol and (datos.promedio.iloc[i] - datos.promedio.iloc[i-1])*(datos.promedio.iloc[i] - datos.promedio.iloc[i+1]) > 0:
                #datos.promedio.iloc[i] = (datos.promedio.iloc[i-1] + datos.promedio.iloc[i+1])/2
                contador_ndwi[i][e] += 1     
                

PUNTOS CON MÁS OUTLIERS

In [15]:
print("Outliers \ttol="+str(tol)+"\t"+prefijolectura)
print(" ")
print("Punto\tmndwi\tndwi")
outliers=[]
for e in rangodefilas:
    sume_mndwi=0
    sume_ndwi=0
    for i in range(len(listadefechas)):
        sume_mndwi += contador_mndwi[i][e]
        sume_ndwi  += contador_ndwi[i][e]        
    outliers.append([sume_mndwi,sume_ndwi])
    if 20<max(outliers[e]):
        print(str(e)+"\t"+str(outliers[e][0])+"\t"+str(outliers[e][1]))

Outliers 	tol=0.25	Vera_
 
Punto	mndwi	ndwi
5	22	23
8	24	11
10	43	35
12	18	23
15	24	12
20	8	25
21	62	59
23	41	44
24	19	40
25	22	21
29	13	46
30	59	36
49	47	50
50	66	92
51	75	64


In [18]:
print("OUTLIERS POR PUNTO")
print("Punto\tmndwi\tndwi")
for e in rangodefilas:
    print(str(e)+"\t"+str(outliers[e][0])+"\t"+str(outliers[e][1]))

OUTLIERS POR PUNTO
Punto	mndwi	ndwi
0	10	11
1	11	9
2	7	6
3	11	8
4	7	8
5	22	23
6	10	8
7	9	6
8	24	11
9	9	7
10	43	35
11	8	10
12	18	23
13	8	8
14	9	11
15	24	12
16	13	9
17	8	10
18	11	9
19	9	11
20	8	25
21	62	59
22	12	14
23	41	44
24	19	40
25	22	21
26	9	7
27	13	10
28	17	12
29	13	46
30	59	36
31	7	7
32	7	7
33	8	6
34	8	7
35	8	7
36	9	10
37	11	10
38	10	8
39	7	6
40	9	6
41	10	8
42	10	10
43	8	7
44	11	12
45	9	7
46	16	18
47	10	10
48	15	12
49	47	50
50	66	92
51	75	64
52	9	6
53	12	10


In [16]:
# Identificador, dada una fecha, nos devuelve los 0 ó 1.
respu=[]
for e in rangodefilas:
    if contador_mndwi[listadefechas.index("2017041")][e]==1:
        respu.append(e)
respu.sort()
print(respu)

[11]


In [19]:
print("Estas son las fechas con más de 10 outliers.")
# Revisando los gráficos, la mayoría tenía nubes sobre Vera.
print("Outliers \ttol="+str(tol)+"\t"+prefijolectura)
print(" ")
print("Fecha\t\t#mndwi\t\t#ndwi")
for i in range(len(listadefechas)):
    if 10<max(sum(contador_mndwi[i]),sum(contador_ndwi[i])):
        print(str(listadefechas[i])+"\t\t"+str(sum(contador_mndwi[i]))+"\t\t"+str(sum(contador_ndwi[i])))

Estas son las fechas con más de 10 outliers.
Outliers 	tol=0.25	Vera_
 
Fecha		#mndwi		#ndwi
2010137		25		22
2010185		12		14
2011049		8		15
2011105		42		43
2013185		23		19
2014137		37		36
2015329		11		8
2016049		41		39
2017249		23		20
2018265		54		1
2019105		54		1
2019161		45		44
